In [26]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
import time
import re

In [27]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [28]:
url = "https://www.imdb.com/chart/top"
soup = getAndParseURL(url)
movie_urls = []
for i in soup.tbody.findAll('tr'):
    movie_urls.append('https://www.imdb.com'+i.findAll('td')[1].a['href'])

In [60]:
row = []
count = 0
for i in movie_urls:
    
    count += 1
    html = getAndParseURL(i)

    try:
        title = html.find('h1', {"data-testid":"hero-title-block__title"}).text
    except:
        title = None
        print("Couldn't scrape the title data")

    try:
        wwgross = int(''.join(html.find('li', {"data-testid":"title-boxoffice-cumulativeworldwidegross"}).text.split('$')[1].split(',')))
    except:
        wwgross = None
        print("Couldn't scrape the gross data")

    try:
        budget = int(re.findall("\d+", ''.join(html.find('li', {"data-testid":"title-boxoffice-budget"}).li.span.text.split(' ')[0].split(',')))[0])
    except:
        budget = None
        print("Couldn't scrape the budget data")

    try:
#         releaseDate = pd.to_datetime('-'.join(html.find('li', {"data-testid":"title-details-releasedate"}).li.a.text.split(' ')[:3]))
        releaseDate = pd.to_datetime(html.find('li', {"data-testid":"title-details-releasedate"}).li.a.text.split('(')[0].strip())
    except:
        releaseDate = None
        print("Couldn't scrape the date data")

    try:
        certificate = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[1].span.text
    except:
        certificate = None
        print("Couldn't scrape the certificate data")

    try:
        runtime = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[2].text.split(' ')
        runtime = int(runtime[0].split('h')[0]) * 60 + int(runtime[1].split('m')[0])
    except:
        runtime = None
        print("Couldn't scrape the runtime data")

    try:
        score = float(html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating__score"}).text.split('/')[0])
    except:
        score = None
        print("Couldn't scrape the score data")

    try:
        scoreN = html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating"}).a.div.div.findAll('div')[1].findAll('div')[2].text
        if 'M' in scoreN:
            scoreNumber = int(float(scoreN.split('M')[0])*1_000_000)
        elif 'K' in scoreN:
            scoreNumber = int(float(scoreN.split('K')[0])*1_000)
        else:
            scoreNumber = int(scoreN)
    except:
        scoreNumber = None
        print("Couldn't scrape the score number data")

    try:
        iR = html.find('ul', {"data-testid":"reviewContent-all-reviews"}).findAll('li')[0].span.span.text
        if 'K' in iR:
            imdbReviews = int(float(iR.split('K')[0])*1_000)
        else:
            imdbReviews = int(iR)
    except:
        imdbReviews = None
        print("Couldn't scrape the imdb review data")

    try:
        criticReviews = int(html.find('ul', {"data-testid":"reviewContent-all-reviews"}).findAll('li')[1].span.span.text)
    except:
        criticReviews= None
        print("Couldn't scrape the critic review data")

    try:
        metaScore = int(html.find('ul', {"data-testid":"reviewContent-all-reviews"}).findAll('li')[2].span.span.text)
    except:
        metaScore= None
        print("Couldn't scrape the meta score data")

    try:
        directors = [] # director = html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[0].text.split('Director')[1]
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[0].findAll('a'):
            directors.append(i.text)
    except:
        directors = None
        print("Couldn't scrape the directors data")

    try:
        writers = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[1].findAll('a'):
            writers.append(i.text)
    except:
        writers = None
        print("Couldn't scrape the writers data")

    try:
        stars = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[2].find('ul', {"role":"presentation"}).findAll('a'):
            stars.append(i.text)
    except:
        stars = None
        print("Couldn't scrape the stars data")

    try:
        genres = []
        for i in html.find('li', {"data-testid":"storyline-genres"}).findAll('li'):
            genres.append(i.text)
    except:
        genres = None
        print("Couldn't scrape the genres data")

    try:
        country = html.find('li', {"data-testid":"title-details-origin"}).a.text
    except:
        country = None
        print("Couldn't scrape the country data")
    
    try:
        locations = []
        for i in html.find('li', {"data-testid":"title-details-filminglocations"}).findAll('a'):
            locations.append(i.text)
        locations = locations[1:-1]
    except:
        locations = None
        print("Couldn't scrape the locations data")

    try:
        companies = []
        for i in html.find('li', {"data-testid":"title-details-companies"}).findAll('a'):
            companies.append(i.text)
        companies = companies[1:-1]
    except:
        companies = None
        print("Couldn't scrape the companies data")

    row.append([title, wwgross, budget, releaseDate, certificate, runtime, score, scoreNumber, imdbReviews,
                   criticReviews, metaScore, directors, writers, stars, genres, country, locations, companies])
    
    print(count, ": ", title, "...Done✔")
    print("Sleeping 5 seconds...")
    time.sleep(5)

column = ["Title", "WorldWideGross", "Budget", "ReleaseDate", "Certificate", "Runtime", "Score",
          "ScoreReviews", "ImdbReviews", "CriticReviews", "MetaScore", "Directors", "Writers",
          "Stars", "Genres", "Country", "Locations", "Companies"]
dfMovies = pd.DataFrame.from_records(row, columns=column)

The Shawshank Redemption ...Done✔
Sleeping 5 seconds...
The Godfather ...Done✔
Sleeping 5 seconds...
The Godfather: Part II ...Done✔
Sleeping 5 seconds...
The Dark Knight ...Done✔
Sleeping 5 seconds...
12 Angry Men ...Done✔
Sleeping 5 seconds...
Schindler's List ...Done✔
Sleeping 5 seconds...
The Lord of the Rings: The Return of the King ...Done✔
Sleeping 5 seconds...
Pulp Fiction ...Done✔
Sleeping 5 seconds...
Il buono, il brutto, il cattivo ...Done✔
Sleeping 5 seconds...
The Lord of the Rings: The Fellowship of the Ring ...Done✔
Sleeping 5 seconds...
Fight Club ...Done✔
Sleeping 5 seconds...
Forrest Gump ...Done✔
Sleeping 5 seconds...
Inception ...Done✔
Sleeping 5 seconds...
The Lord of the Rings: The Two Towers ...Done✔
Sleeping 5 seconds...
Star Wars: Episode V - The Empire Strikes Back ...Done✔
Sleeping 5 seconds...
The Matrix ...Done✔
Sleeping 5 seconds...
Goodfellas ...Done✔
Sleeping 5 seconds...
One Flew Over the Cuckoo's Nest ...Done✔
Sleeping 5 seconds...
Shichinin no samurai

El laberinto del fauno ...Done✔
Sleeping 5 seconds...
A Beautiful Mind ...Done✔
Sleeping 5 seconds...
El secreto de sus ojos ...Done✔
Sleeping 5 seconds...
Raging Bull ...Done✔
Sleeping 5 seconds...
Couldn't scrape the meta score data
Dersu Uzala ...Done✔
Sleeping 5 seconds...
Chinatown ...Done✔
Sleeping 5 seconds...
Couldn't scrape the locations data
Tonari no Totoro ...Done✔
Sleeping 5 seconds...
Lock, Stock and Two Smoking Barrels ...Done✔
Sleeping 5 seconds...
Shutter Island ...Done✔
Sleeping 5 seconds...
No Country for Old Men ...Done✔
Sleeping 5 seconds...
Couldn't scrape the meta score data
The Gold Rush ...Done✔
Sleeping 5 seconds...
Dial M for Murder ...Done✔
Sleeping 5 seconds...
The Thing ...Done✔
Sleeping 5 seconds...
Det sjunde inseglet ...Done✔
Sleeping 5 seconds...
Three Billboards Outside Ebbing, Missouri ...Done✔
Sleeping 5 seconds...
Couldn't scrape the gross data
Couldn't scrape the locations data
Klaus ...Done✔
Sleeping 5 seconds...
The Elephant Man ...Done✔
Sleepin

In [61]:
dfMovies.to_csv("data/movies.csv")
dfMovies

,Title,WorldWideGross,Budget,ReleaseDate,Certificate,Runtime,Score,ScoreReviews,ImdbReviews,CriticReviews,MetaScore,Directors,Writers,Stars,Genres,Country,Locations,Companies
0,The Shawshank Redemption,2.881729e+07,25000000.0,1995-03-10,13+,142.0,9.3,2500000,9700,193,80.0,[Frank Darabont],"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",[Drama],United States,"[127A Smithfield Road, Frederiksted, Virgin Is...",[Castle Rock Entertainment]
1,The Godfather,2.461210e+08,6000000.0,1973-10-01,18+,175.0,9.2,1700000,4700,269,100.0,[Francis Ford Coppola],"[Mario Puzo, Francis Ford Coppola]","[Marlon Brando, Al Pacino, James Caan]","[Crime, Drama]",United States,"[Forza d'Agrò, Messina, Sicily, Italy]","[Paramount Pictures, Alfran Productions]"
2,The Godfather: Part II,4.803578e+07,13000000.0,1974-12-18,18+,202.0,9.0,1200000,1200,190,90.0,[Francis Ford Coppola],"[Francis Ford Coppola, Mario Puzo]","[Al Pacino, Robert De Niro, Robert Duvall]","[Crime, Drama]",United States,"[Kaiser Estate, 4000 W Lake Blvd, Homewood, La...","[Paramount Pictures, The Coppola Company, Amer..."
3,The Dark Knight,1.005974e+09,185000000.0,2008-07-25,16+,152.0,9.0,2500000,7700,433,84.0,[Christopher Nolan],"[Writers, Jonathan Nolan, Christopher Nolan, D...","[Christian Bale, Heath Ledger, Aaron Eckhart]","[Action, Crime, Drama, Thriller]",United States,"[Chicago, Illinois, USA]","[Warner Bros., Legendary Entertainment, Syncopy]"
4,12 Angry Men,9.550000e+02,350000.0,1960-04-01,15+,96.0,9.0,740000,1800,160,96.0,[Sidney Lumet],[Reginald Rose],"[Henry Fonda, Lee J. Cobb, Martin Balsam]","[Crime, Drama]",United States,[New York County Courthouse - 60 Centre Street...,[Orion-Nova Productions]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Le notti di Cabiria,7.553190e+05,NaN,1957-05-10,Not Rated,110.0,8.1,47000,119,105,NaN,[Federico Fellini],"[Writers, Federico Fellini, Ennio Flaiano, Tul...","[Giulietta Masina, François Périer, Franca Marzi]",[Drama],Italy,"[Castel Gandolfo, Rome, Lazio, Italy]","[Dino de Laurentiis Cinematografica, Les Films..."
246,The Princess Bride,3.090244e+07,16000000.0,1987-09-25,PG,98.0,8.1,413000,926,227,77.0,[Rob Reiner],[William Goldman],"[Cary Elwes, Mandy Patinkin, Robin Wright]","[Adventure, Family, Fantasy, Romance]",United Kingdom,"[Cliffs of Moher, County Clare, Ireland]","[Act III Communications, Buttercup Films Ltd.,..."
247,"Paris, Texas",2.207548e+06,1162000.0,2020-10-02,13A,145.0,8.1,101000,286,110,78.0,[Wim Wenders],"[L.M. Kit Carson, Sam Shepard, Walter Donohue]","[Harry Dean Stanton, Nastassja Kinski, Dean St...",[Drama],West Germany,"[Marathon Motel & RV Park - Highway 90, Marath...","[Road Movies Filmproduktion, Argos Films, West..."
248,Trois couleurs: Rouge,3.581969e+06,NaN,1995-02-03,15+,99.0,8.1,99000,184,79,100.0,[Krzysztof Kieslowski],"[Writers, Krzysztof Kieslowski, Krzysztof Pies...","[Irène Jacob, Jean-Louis Trintignant, Frédériq...","[Drama, Mystery, Romance]",Switzerland,"[Rue des Sources, Geneva, Canton de Genève, Sw...","[MK2 Productions, France 3 Cinéma, CAB Product..."
